# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.26
Project name: dehergne
Project home: /Users/jrc/mhk-home/sources/dehergne
Database type: sqlite
Database name: dehergne
Kleio image: timelinkserver/kleio-server
Kleio server token: fQOU0...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/mhk-home/sources/dehergne
Kleio server container: friendly_elion
Kleio version requested: latest
Kleio server version: 12.9.588 (2025-06-06 16:04:22)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne, project_home=/Users/jrc/mhk-home/sources/dehergne, db_type=sqlite, db_name=dehergne, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,28
1,alembic_version,1
2,aregisters,0
3,attributes,26819
4,blinks,45
5,class_attributes,58
6,classes,10
7,entities,32957
8,geoentities,359
9,kleiofiles,29


### Show the kleio files available 

In [13]:
kleio_files = tlnb.get_kleio_files()
kleio_files[['name','status','errors','warnings','import_status','import_status','import_errors']]


,name,status,errors,warnings,import_status,import_status,import_errors
0,dehergne-0-abrev.cli,V,0,0,I,I,0
1,dehergne-a.cli,V,0,0,I,I,0
2,dehergne-b.cli,V,0,0,I,I,0
3,dehergne-c.cli,V,0,0,N,N,0
4,dehergne-d.cli,V,0,0,I,I,0
5,dehergne-e.cli,V,0,0,I,I,0
6,dehergne-f.cli,V,0,0,I,I,0
7,dehergne-g.cli,V,0,0,I,I,0
8,dehergne-h.cli,V,0,0,I,I,0
9,dehergne-i.cli,V,0,0,I,I,0


## Optionally clean translations 

Do this if there was an update to the Kleio server and you want to make sure that all translations are redone to incorporate the changes.

In [4]:
# NBVAL_IGNORE_OUTPUT

# tlnb.kleio_server.translation_clean("", recurse="yes")


### Translate and import as needed

In [14]:

tlnb.db.update_from_sources(path='', with_import_errors=True)

/Users/jrc/develop/timelink-py/timelink/api/models/pom_som_mapper.py:698: UserWarning: Ambiguous mapping! more than one element of rel is mapped to column class date check mapping for this group
  element: KElement = group.get_element_by_name_or_class(cattr.colclass)


## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

### Check a translation report

If needed to check for errors.


In [5]:

# get the import status
imported_files_df = tlnb.get_import_status()
errors = imported_files_df[(imported_files_df["errors"] > 0) | (imported_files_df["import_errors"] > 0)]
errors[["name", "errors", "warnings", "import_status", "import_errors", "import_warnings", "path"]].sort_values("name")

,name,errors,warnings,import_status,import_errors,import_warnings,path


In [6]:
file_number=list(errors.index.unique())
if len(file_number) > 0:
    rpt = tlnb.get_translation_report(imported_files_df, file_number)
    print(rpt)
else:
    print("No errors found")



No errors found


### Get the import report for a file

In case there are errors in the import phase check the import report.

In [7]:
ierrors = imported_files_df[imported_files_df.import_errors>0][["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")
ierrors

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path


In [8]:

rpt = tlnb.get_import_rpt(imported_files_df, rows=list(ierrors.index), match_path=True)
print(rpt)

In [9]:
tlnb.get_import_rpt('dehergne-a.cli')  # just the file name of the kleio file

'No errors\nNo warnings'